In [11]:

%matplotlib widget
import rospy
import ipywidgets as widgets
import actionlib
import actionlib.msg
import assignment_2_2023.msg
import matplotlib.pyplot as plt
#%matplotlib inline
import numpy as np


from assignment_2_2023.srv import LastTarget, LastTargetRequest
from nav_msgs.msg import Odometry
from ipywidgets import Layout
from matplotlib.animation import FuncAnimation


last_positions_x=[]
last_positions_y=[]

setted_target_x=[]
setted_target_y=[]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [2]:
def on_Send_Goal_Button_Clicked(b):
    client.wait_for_server()
    goal = assignment_2_2023.msg.PlanningGoal()
    goal.target_pose.pose.position.x = des_x.value
    goal.target_pose.pose.position.y = des_y.value
    setted_target_x.append(des_x.value)
    setted_target_y.append(des_y.value)
    #print(goal)
    client.send_goal(goal)
    

In [3]:
def on_Cancel_Button_Clicked(b):
    last_positions_x.append(des_x.value)
    last_positions_y.append(des_y.value)
    
    client.cancel_goal()



In [4]:

des_x=widgets.BoundedFloatText(min=-9, max=9.0,description='x position')
display(des_x)

des_y=widgets.BoundedFloatText(min=-9, max=9.0, description='y position')
display(des_y)
rospy.init_node('action_client')
client = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2023.msg.PlanningAction)

sendGoal = widgets.Button(description="Send Goal !") 
cancelGoal=widgets.Button(description="Cancel Goal !")
sendGoal.on_click(on_Send_Goal_Button_Clicked)
cancelGoal.on_click(on_Cancel_Button_Clicked)
display(sendGoal)
display(cancelGoal)





BoundedFloatText(value=0.0, description='x position', max=9.0, min=-9.0)

BoundedFloatText(value=0.0, description='y position', max=9.0, min=-9.0)

Button(description='Send Goal !', style=ButtonStyle())

Button(description='Cancel Goal !', style=ButtonStyle())

In [5]:
def clbk_odom(msg):
# Callback function that process incoming Odometry messages,
# extract informations about: position x, position y, linear velocity x and
# angular velocity z; Save these informations in a custom message Info()
# and pubblish that informations on /robot_pos_vel topic.

# Initialize a new message
# The struct of Info() is (x,y,vel_x,vel_z)
    global pos_x,pos_y,vel_x,vel_z

    pos_x = msg.pose.pose.position.x
    pos_y = msg.pose.pose.position.y
    vel_x = msg.twist.twist.linear.x 		# linear velocity along x axis
    vel_z = msg.twist.twist.angular.z		# angular velocity around z axis
   
    

    
    


In [6]:

sub = rospy.Subscriber('/odom', Odometry, clbk_odom)
print(f'pos_x:{pos_x} pos_y:{pos_y} vel_x:{vel_x}')

style = {'description_width': 'initial'}
current_x=widgets.Text(
              value=str(pos_x),
              description="current position x",
              style=style
         )
current_y=widgets.Text(
                    value=str(pos_y),
                    description="current posizion y",
                    style=style
        ) 
current_velx=widgets.Text(
                  value=str(vel_x),
                  description="current position x",
                  style=style    
        )
current_velz=widgets.Text(
                    value=str(vel_z),
                    description="current posizion y",
                    style=style
        ) 

display(current_x)
display(current_y)
display(current_velx)
display(current_velz)


#last_positions_x
x=widgets.Text(
    value=str(setted_target_x),
    description="last position x"
    )
y=widgets.Text(
    value=str(last_positions_y),
    description="last posizion y"
    )
display(x)
display(y)



NameError: name 'pos_x' is not defined

In [7]:
setted_target_x
setted_target_y

[]

In [8]:

class Visualizer:
    def __init__(self):
        self.fig,self.ax=plt.subplots()
        self.ln=plt.plot([],[],'ro')
        self.x_data,self.y_data=[],[]
    
    def plot_init(self):
        self.ax.set_xlim(-10,10)
        self.ax.set_ylim(-10,10)
        return self.ln,
    
    def clback_odom(self,msg):
        #global pos_x,pos_y,vel_x,vel_z
        self.x_data.append(msg.pose.pose.position.x)
        self.y_data.append(msg.pose.pose.position.y)
    
    #vel_x = msg.twist.twist.linear.x 		# linear velocity along x axis
    #vel_z = msg.twist.twist.angular.z		# angular velocity around z axis
   
    def update_plot(self,frame):
        plt.plot(self.x_data,self.y_data)
        return self.ln,

    
            

In [10]:
#rospy.init_node('Visualizer')

vis=Visualizer()
sub = rospy.Subscriber('/odom', Odometry, vis.clback_odom)
ani=FuncAnimation(vis.fig,vis.update_plot,init_func=vis.plot_init,interval=50)
plt.show(block=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …